In [ ]:
# setup en google colab
# descoméntalo si estás en dicho entorno
# !git clone https://github.com/koldLight/python-data-ull
# %cd python-data-ull

# Pivotación de tablas

Vamos a ver cómo transformar las tablas de formato ancho a largo y viceversa

In [ ]:
import pandas as pd

In [ ]:
air = pd.read_csv("dat/airquality.csv")
air.head()

## Melt: de ancho a largo

Para pasar de formato ancho a largo, podemos usar [`melt`](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.melt.html)

In [ ]:
air_long = air.melt(id_vars=['month', 'day'])
air_long.head()

In [ ]:
len(air_long)

Vemos que, para cada mes y día, ahora contamos con dos columnas: la variable medida y su valor.

En el formato largo, cada fila cuenta con el índice (en este caso, mes y día), un valor, y etiquetas del valor.

## Pivot: de largo a ancho

Para pasar de formato largo a ancho, podemos usar [`pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

In [ ]:
air_wide = air_long.pivot_table(index=['month', 'day'], columns='variable', values='value')
air_wide.head()

Los índices jerárquicos suelen ser incómodos para tratar la tabla. Podemos quitarlo con `reset_index()`

In [ ]:
air_wide = air_wide.reset_index()
air_wide.head()

In [ ]:
# Si además queremos quitar el nombre "feo" que se ha quedado como nombre de las columnas...
air_wide.columns.name = ''
air_wide.head()

#### Ejercicio

Investiga y cambia los nombres de las columnas resultantes (variable y value) por otros que consideres adecuados, durante el proceso de `melt`.

#### Ejercicio

La función [`pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) permite agregar si hay más de un valor por cada fila (es decir, parejas de índice + variable duplicadas).

Aprovecha esta utilidad para, en el paso de `air_long` a ancho, calcular la mediana por variable por mes.

#### Ejemplo del mundo real

Examina los datos contenidos en el fichero `dat/contaminacion_mad_201812.csv`, y mira qué aspecto tiene. Nuestro objetivo es dejarlo en formato largo, y quedarnos únicamente con los días con una medida válida.

Para hacerlo, podemos dividir el problema en los siguientes pasos:

* A partir de ese dataframe, crear dos nuevos:

    * Uno con el índice + las columnas con los valores (D01, ..., D31)
    * Otro con el índice + las columnas con la validez (V01, ..., V31)

* Pasar ambos a formato largo. Es decir, queremos tener una columna `dia` y otra `validez` y que los valores pasen a ser filas, en lugar de tener 31 columnas. El día debe quedar como numérico en ambas tablas.
* Cruzar los resultados, para tener en el mismo dataframe valor y validez.
* Limpiar la fecha, para tenerla como `date`.
* Castear la validez a booleano (True / False).

In [ ]:
dat = pd.read_csv("dat/contaminacion_mad_201812.csv", sep=";")
dat.head()

In [ ]:
# Separando en dos dataframes
# Uno con el índice + las columnas con los valores (D01, ..., D31)
# Otro con el índice + las columnas con la validez (V01, ..., V31)
cols_indice = list(dat.columns[:7])
cols_valores = list(dat.columns[7::2])
cols_validez = list(dat.columns[8::2])

dat_valores = dat[cols_indice + cols_valores]
dat_validez = dat[cols_indice + cols_validez]
dat_validez.head()

In [ ]:
# Pasamos a formato largo
dat_valores_largo = dat_valores.melt(id_vars=cols_indice, var_name='DIA', value_name='VALOR')
dat_validez_largo = dat_validez.melt(id_vars=cols_indice, var_name='DIA', value_name='VALIDEZ')

# Queremos convertir D01, ... y V01, ... a 1..31
dat_valores_largo["DIA"] = dat_valores_largo["DIA"].str.replace('D', '').astype(int)
dat_validez_largo["DIA"] = dat_validez_largo["DIA"].str.replace('V', '').astype(int)
dat_valores_largo.head()

In [ ]:
# Cruzamos ambos dataframe para tenerlo todo en el mismo
dat_largo = dat_valores_largo.merge(dat_validez_largo)
dat_largo.head()

In [ ]:
from datetime import date

# Creación de la fecha
# Como hay fechas que no existen como 30-feb, encapsulo en try / except
def try_create_date(fila):
    try:
        return date(fila.ANO, fila.MES, fila.DIA)
    except:
        return None

# Crear una columna de fecha (date)
dat_largo["FECHA"] = dat_largo.apply(try_create_date, axis=1)

# Y ahora elimino las fechas None (esos 30-feb, etc)
dat_largo = dat_largo[~dat_largo.FECHA.isnull()].copy()

# Casting de VALIDEZ a boolean
dat_largo["VALIDEZ"] = dat_largo["VALIDEZ"] == "V"
dat_largo.head()

In [ ]:
# Podríamos querer guardar el resultado ya limpio
# dat_largo.to_csv("contaminacion_limpio.csv")

#### Referencia

Puedes consultar la guía completa de pandas sobre pivotación [aquí](http://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)